In [1]:
from pymongo import MongoClient
import json
from bson import ObjectId
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pandas as pd
import unicodedata
import os
import requests 

from dotenv import load_dotenv
from utils import espiadinha, get_df
load_dotenv()

True

In [2]:
#credenciais de acesso ao alexandria
server_type = os.getenv("server_type")
MONGO_ALEXANDRIA_USERNAME = os.getenv('MONGO_ALEXANDRIA_USERNAME')
MONGO_ALEXANDRIA_PASSWORD = os.getenv('MONGO_ALEXANDRIA_PASSWORD')
MONGO_ALEXANDRIA_HOST = os.getenv('MONGO_ALEXANDRIA_HOST')
str_conn = f"{server_type}://{MONGO_ALEXANDRIA_USERNAME}:{MONGO_ALEXANDRIA_PASSWORD}@{MONGO_ALEXANDRIA_HOST}" 
client = MongoClient(str_conn)
db = client['alexandria-kroton']

In [4]:
def from_html_to_text(tag):
    soup = BeautifulSoup(tag, features="html.parser")
    for script in soup(["script", "style"]):
        script.extract()   
    text = soup.get_text()
    return text

In [3]:
#import das bases do alexandria
cursos={
        'Engenharia':['ADMINISTRAÇÃO E ECONOMIA PARA ENGENHEIROS','DESIGN THINKING E INOVAÇÃO DOS MODELOS DE NEGÓCIOS',
                            'ENGENHARIA, CIÊNCIA E TECNOLOGIA','LEGISLAÇÃO, SEGURANÇA DO TRABALHO E MEIO AMBIENTE',
                            'CÁLCULO DIFERENCIAL E INTEGRAL II','FÍSICA GERAL E EXPERIMENTAL - ENERGIA',
                            'DESENHO TÉCNICO PROJETIVO','MÉTODOS MATEMÁTICOS']
    }

df_disc=get_df(db,'disciplines',['title','learningUnits','krotonId'])
df_LUs=get_df(db,'learningunits',['_id','classes','krotonId'])
df_cls=get_df(db,'classes',['_id','blocks','krotonId','title'])
df_blk=get_df(db,'blocks',['_id','questions','content','title'])
df_quest=get_df(db,'questions',['_id','type','questionModel','title','description','feedback','alternatives','classId'])

In [199]:
sequencia=0
curso_lista=['Engenharia']
for curso in curso_lista:
    disciplina = cursos[curso][sequencia]
    disciplina = ''.join(c for c in disciplina if not unicodedata.category(c).startswith('C'))
    print(f"Requisitando curso: {curso} - Disciplina: {disciplina}")
    
    #Filtro de Unidades de ensino de uma Disciplina
    filtro_LUs = df_disc[df_disc.title.isin([disciplina])]['learningUnits'].to_list()[-1]
    ID_MAIA_API_ORDER = df_disc[df_disc.title.isin([disciplina])]['krotonId'].item()
    url = f"https://sag-api-kroton.platosedu.io/api/v1/external/components/{ID_MAIA_API_ORDER}/learningUnits"
    ordem = requests.request("GET", url, data="",headers={'x_secret': os.getenv('MONGO_MAIA_PASSWORD')})
    ordem = json.loads(ordem.text)
    df_ordem = pd.DataFrame()
    for row in range(len(ordem)):
        df_ordem=pd.concat([df_ordem,pd.DataFrame([ordem[row]['id'],ordem[row]['order'],ordem[row]['title']]).transpose()])
    df_ordem.columns=['krotonId','ORDER','LU_TITLE']

    #Filtro e enriquecimento da base LUs com ordem de Maia
    df_LUs_filtrada = df_LUs[df_LUs._id.isin(filtro_LUs)].merge(df_ordem,how='left',on='krotonId')
    #Filtro de classes dentro de uma unidade de ensino
    filtro_LU_CLASSES_aninhado = df_LUs_filtrada.sort_values('ORDER')['classes'].to_list()
    
    #Para cada unidade de ensino vamos extrair o conteúdo de cada aula
    dict_conteudo = {}
    for LU_ORDER, filtro_CLASSES_FROM_LU in enumerate(filtro_LU_CLASSES_aninhado):
        LU_ORDER=LU_ORDER+1
        #print(f'Fazendo LU: {LU_ORDER}')
        
        #filtro assets são todos os assets de cada aula de uma unidade de ensino
        filtro_ASSETS = df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)]['blocks'].to_list()
        TITULO_LU = df_LUs_filtrada[df_LUs_filtrada.ORDER==LU_ORDER]['LU_TITLE'].item()
        dict_conteudo[f"Unidade_{LU_ORDER}__{TITULO_LU}"] = {}
        #
        for AULA_ORDER, ASSETS_FROM_AULA in enumerate(filtro_ASSETS):
            AULA_ORDER=AULA_ORDER+1
            #print(f'Fazendo aula: {AULA_ORDER}')
            TITULO_AULA =df_cls[df_cls._id.isin(filtro_CLASSES_FROM_LU)].iloc[AULA_ORDER-1]['title']
            conteudo = f"Unidade_{LU_ORDER} - AULA_{AULA_ORDER} - {TITULO_AULA} =|> "
            for texto in df_blk[
                (df_blk._id.isin(ASSETS_FROM_AULA))
                &(df_blk.content.notna())
                &(~df_blk.title.isin(['Siga em Frente...','Referências','Saiba mais']))
                ][['title','content']].iterrows():
                conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
            dict_conteudo[f"Unidade_{LU_ORDER}__{TITULO_LU}"][f"AULA_{AULA_ORDER}__{TITULO_AULA}"] = conteudo
    


Requisitando curso: Engenharia - Disciplina: ADMINISTRAÇÃO E ECONOMIA PARA ENGENHEIROS


C:\Users\renan.vital\AppData\Local\Temp\ipykernel_11816\44074283.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_11816\44074283.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteudo=conteudo+f'{from_html_to_text(texto[1][0])}: {from_html_to_text(texto[1][1])}'
C:\Users\renan.vital\AppData\Local\Temp\ipykernel_11816\44074283.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labe

In [156]:
index_aulas = [Aula for Unidade in dict_conteudo.keys() for Aula in dict_conteudo[Unidade].keys()]
index_aulas

['AULA_1__Origem do desenho técnico',
 'AULA_2__Normas ABNT',
 'AULA_3__Utilização de Instrumentos de Desenho',
 'AULA_4__Desenho Universal',
 'AULA_5__Fundamentos do Desenho Técnico',
 'AULA_1__Geometria Descritiva ',
 'AULA_2__Desenho Projetivo',
 'AULA_3__Projeção Ortogonal',
 'AULA_4__Perspectiva ',
 'AULA_5__Geometria Descritiva e Projeções',
 'AULA_1__Introdução ao desenho auxiliado por computador',
 'AULA_2__Camadas de trabalho',
 'AULA_3__Comandos de Desenho',
 'AULA_4__Ferramentas de visualização, divisão de elementos e coleta de dados',
 'AULA_5__Desenho Auxiliado por Computador: Conceitos Básicos',
 'AULA_1__Comandos básicos de edição',
 'AULA_2__Comandos avançados de edição',
 'AULA_3__Ferramentas de auxílio ao desenho, cotagem e aplicação de hachuras',
 'AULA_4__Otimização e impressão de projetos',
 'AULA_5__Desenho Auxiliado por Computador: Ferramentas Complementares e Apresentação Gráfica']

In [55]:
lista_kcs = pd.read_parquet('lista_kcs.parquet')
lista_kcs.head()

,KC,disciplina
0,Conhecimentos numéricos - operações em conj...,Matematica
1,Conhecimentos geométricos - características...,Matematica
2,"Conhecimentos geométricos - grandezas, unid...",Matematica
3,"Conhecimentos geométricos - comprimentos, á...",Matematica
4,Conhecimentos geométricos - ângulos,Matematica


In [21]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint='https://openai-dados-lab-poc.openai.azure.com/',
    api_key=os.environ["AZURE_OPENAI_API_KEY"]
)   

In [186]:
df_kc = pd.DataFrame()
for Unidade in dict_conteudo.keys():
    for Aula in dict_conteudo[Unidade].keys():
        texto_aula = dict_conteudo[Unidade][Aula]
        
        aux = {}
        vector_store = FAISS(
            embedding_function=embeddings,
            index=faiss.IndexFlatL2(len(embeddings.embed_query("Which item is related to the following lecture?"))),
            docstore=InMemoryDocstore(),
            index_to_docstore_id={})
   
        vector_store.add_documents(
            documents=[Document(i) for i in lista_kcs.KC],
            ids = [str(id) for id in range(len(lista_kcs.KC.to_list()))]
        )
        results = vector_store.similarity_search_with_relevance_scores(texto_aula,
                                                 k=10,
                                                 search_type='similarity_score_threshold',
                                                 score_threshold=0.8)
        kc = [kc[0].page_content for kc in results] 
        similarity = [kc[1] for kc in results]       
        aux['Unidade']=str(Unidade)
        aux['Aula']=str(Aula)
        aux['KC']=kc
        aux['Similarity']=similarity
        df_kc = pd.concat([df_kc,pd.DataFrame([aux])])
        
#df_kc

No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score threshold 0.8
No relevant docs were retrieved using the relevance score thresh

In [179]:
dict_conteudo['Unidade_3__Conceitos Gerais e Fundamentos sobre a Economia'].keys()

dict_keys(['AULA_1__Contextualização Histórica sobre Economia', 'AULA_2__Fundamentos gerais relacionados à Economia', 'AULA_3__Introdução à Microeconomia', 'AULA_4__Fundamentos da Matemática Financeira', 'AULA_5__Conceitos Gerais e Fundamentos sobre a Economia'])

In [182]:
aula_texto_exemplo = dict_conteudo['Unidade_3__Conceitos Gerais e Fundamentos sobre a Economia']['AULA_4__Fundamentos da Matemática Financeira']

'Unidade_3 - AULA_4 - Fundamentos da Matemática Financeira =|> Ponto de Partida: Olá, estudante! Boas-vindas a você em mais uma aula! Vamos iniciar com um assunto que mexe com todos: dinheiro! Afinal, como consumidores, precisamos controlar financeiramente o nosso orçamento, e o mesmo acontece com as empresas, elas também precisam controlar o seu fluxo de caixa, uma vez que o risco de perdas em um espaço curto de tempo é grande.Nessa realidade, vamos conhecer um pouco a história de Davi, jovem estudante de engenharia que trabalhou nos últimos anos como representante comercial em uma concessionária de veículos, mas, por ajustes financeiros, foi dispensado há 6 meses. Após a sua saída,, Davi abriu uma fábrica de carteiras de couro cujo nome era CartCow. O início dessa jornada está complicada, pois ele está realizando a gestão financeira sozinho: pagamento dos funcionários, pagamento dos fornecedores, pagamento dos impostos, de forma que todas as contas do dia a dia da CartCow tornaram-se uma rotina para Davi. Basicamente, desde que abriu a empresa, o engenheiro necessita utilizar o limite do cheque especial, em alguns dias do mês, para que as contas da CartCow fechem no positivo ao final do mês. No entanto, como está pagando juros altos, ele precisa de planejamento financeiro para sair de tal situação, o que o levou a pesquisar em vários bancos a possibilidade da CartCow tomar dinheiro emprestado. Será que Davi conseguirá definir um empréstimo que valerá a pena? Somente um empréstimo será suficiente para que os resultados financeiros da fábrica melhorem? O que Davi deverá fazer para sair dessa situação?\xa0Para auxiliar a Davi, vamos estudar os conceitos a respeito da matemática financeira. Isso o ajudará a tomar a decisão correta. Vamos lá!Fundamentos da matemática financeira: Estudante, esta videoaula foi preparada especialmente para você. Nela, você irá aprender conteúdos importantes para a sua formação profissional. Vamos assisti-la?\xa0Clique aqui para acessar os slides da sua videoaula.Bons estudos!Vamos Exercitar?: Caro estudante, nesta aula trouxemos os conceitos básicos da matemática financeira que auxiliam o gestor a tomar decisões relativas ao seu negócio no cenário financeiro. Dessa maneira, conseguimos entender que a gestão financeira de uma empresa é uma tarefa complexa. Como consequência de os engenheiros estarem assumindo cargos financeiras em organizações, fez-se a necessidade de análises, assim como ocorre com Davi da empresa CartCow.Retornando ao caso apresentado no início desta aula, para que Davi consiga sair da situação financeira difícil que está atravessando, ele deverá elaborar um fluxo de caixa para visualizar se os pagamentos e os recebimentos estão desiquilibrados somente em alguns dias do mês, ou se o seu saldo operacional é negativo, o que indicaria um problema financeiro mais grave. Outra atividade para Davi será calcular o quanto ele está pagando de juros no cheque especial e verificar se existe outra modalidade de empréstimo mais em conta no mercado, inclusive, tendo conhecimento de como lidar com o sistema de amortização que seja compatível com a realidade da CartCow, não esquecendo, também, das diferenças de juros simples e compostos. Com essas ações, ele deverá sair do problema que está enfrentando.É preciso ter muita disciplina!Vamos Começar!: Um dos grandes desafios das empresas é a elaboração e a execução do controle financeiro. Mesmo as empresas de grande porte, as quais possuem profissionais experientes e sistemas avançados, podem se descuidar desse controle e correrem riscos. Portanto, é essencial o entendimento de conceitos e técnicas que afetam diretamente o comportamento das empresas, independentemente do ramo em que atuam, quanto às suas decisões de curto, médio e longo prazos. Dentre os conceitos, destacamos o Fluxo de Caixa, que, de acordo com Neto (2017, p. 141), “representa uma série de pagamentos ou de recebimentos que se estima ocorrer em determinado intervalo de tempo.”É comum que as empresas tenham que lidar com pagamentos e recebimentos ao longo do tempo, sendo necessária a gestão correta e eficiente desses recursos, sob pena de impactar negativamente os seus resultados. De forma geral\xa0os fluxos de caixa podem ser verificados das mais variadas formas e tipos em termos de períodos de ocorrência (postecipados, antecipados ou diferidos), de periodicidade (períodos iguais entre si ou diferentes), de duração (limitados ou indeferidos) e de valores (constantes ou variáveis). (NETO, 2017, p. 141)Destacamos os seguintes termos dos fluxos de caixas: PMT, PV, FV, i e n, sendo que cada um deles representa uma variável:PMT: pagamentos de mesmo valor (recorrentes).PV: valor presente.FV: valor futuro.n: número de períodos.i: taxa de juros.Neto (2017) destaca que a fórmula do valor presente de um fluxo de caixa uniforme é dada por: PV = PMT x FPV (i, n). Ele e apresenta, também, o seguinte exemplo:\xa0Determinado bem é vendido em 7 pagamentos mensais, iguais e consecutivos de $ 4.000,00. Para uma taxa de juros de 2,6% a.m., até que preço compensa adquirir o aparelho à vista?Solução:PMT = $ 4.000,00PV = ?i = 2,6% a.m.n = 7PV = PMT x FPV (i, n)\xa0PV = 4.000 x 6,325294 = $ 25.301,18 (NETO, 2017, p. 145)Observamos que o termo FPV se refere ao número de períodos descontada a taxa em cada um deles, ou seja: 1 – (1+i)^-n / i. No exemplo citado, o número de períodos é 7, de forma que a expressão se traduz em: 1 – (1,026)^-7 / 0,026 = 6,325294. Vale ressaltar que o mesmo raciocínio pode ser aplicado para o cálculo de valor futuro e, portanto, calcular o desconto de pagamentos das taxas de juros pré-fixadas. Portanto, a formulação genérica para o valor futuro é dada por: FV = PMT x FFV (i, n).\xa0 Neto (2017, p. 148) expõe o seguinte exemplo:\xa0Calcular o montante acumulado ao final do 7o mês de uma sequência de 7 depósitos mensais e sucessivos, no valor de $ 800,00 cada, numa conta de poupança que remunera a uma taxa de juros de 2,1% a.m.\xa0Solução:\xa0O valor futuro pode ser calculado pela soma do montante de cada depósito, isto é:FV = 800,00 + 800,00 (1,021) + 800,00 (1,021)^2 + 800,00 (1,021)^3 + ... + 800,00 (1,021)^6FV = $ 5.965,41Aplicando-se a fórmula-padrão de apuração do valor futuro, tem-se, de forma abreviada, o mesmo resultado:FV = 800 x (1 + i)^n -1 / iFV = 800 x (1,021)^7 -1 / 0,021FV = 800 x 7,456763 = $ 5.965,41\xa0Vale citar outra importante variável utilizada para explicar o funcionamento dos fluxos financeiros das empresas. Nesse caso, a taxa de juros, considerada como balizador das decisões de alocação de portfólio dos agentes econômicos. Na economia, a\xa0 taxa de juros\xa0 relaciona-se à demanda por moeda e por crédito, o que afeta as decisões dos empresários quanto à realização de novos investimentos. Além disso, a taxa SELIC, que é a taxa de juros básica da economia brasileira, estabelecida pelo Banco Central, relaciona-se com a meta de inflação, de forma que o nível de preços está intimamente relacionado com a taxa de juros, ou seja,\xa0As taxas de juros nominais constituem um pagamento expresso em percentagem mensal, trimestral, anual etc., que um tomador de empréstimos faz ao emprestador em troca do uso de determinada quantia de dinheiro. Se não houver inflação no período, a taxa de juros nominal será igual à taxa de juros real desse mesmo período de tempo. (VASCONCELOS, 2019, p. 204)\xa0'



In [185]:
df_kc.sample()

,Unidade,Aula,KC,Similarity
0,Unidade_1__Fundamentos da Administração e Cont...,AULA_4__Empresas,[ Conhecimentos algébricos/geométricos - pa...,"[0.7149161035677052, 0.7072081614714874, 0.704..."


In [181]:
linha=13
print(df_kc.iloc[linha].Aula)
print(df_kc.iloc[linha].Similarity)
df_kc.iloc[linha].KC

AULA_4__Fundamentos da Matemática Financeira
[0.7548094654558429, 0.7206312583726309, 0.7161647882485374, 0.716154694078319, 0.7147735207791928, 0.7145368451513798, 0.7138569954114748, 0.7095131304696995, 0.7067038112087817, 0.7053122697849217]


[' Conhecimentos numéricos  -  operações em conjuntos numéricos (naturais, inteir os, racionais e reais), desigualdades, divisibilidade, fatoração, razões e proporções,  porcentagem e juros, relações de dependência entre grandezas, sequências e  progressões, princípios de contagem. ',
 ' Conhecimentos algébricos -   relações no ciclo trigonométrico e funções trigonométricas. ',
 ' Conhecimentos de estatística e probabilidade  -  noções de probabilidade.  ',
 ' Conhecimentos algébricos -  equações e inequações',
 ' Conhecimentos algébricos -  gráficos e funções',
 ' Conhecimentos algébricos -  funções algébricas do 1.º e do 2.º graus, polinomiais, racionais, exponenciais e logarítmicas',
 ' Conhecimentos básicos e fundamentais  -  Observações e mensurações: representação de grandezas físicas como  grandezas mensuráveis',
 ' Conhecimentos básicos e fundamentais  -  Co nceituação de grandezas vetoriais e escalares',
 ' Conhecimentos de estatística e probabilidade  -  desvios e  variância'

Conhecimentos numéricos - operações em conjuntos numéricos (naturais, inteiros, racionais e reais),desigualdades, divisibilidade, fatoração, razões e proporções, porcentagem e juros, relações de dependência entre grandezas, sequências e progressões, princípios de contagem.',
'Conhecimentos algébricos - gráficos e funções',
'Conhecimentos algébricos - funções algébricas do 1.º e do 2.º graus, polinomiais, racionais, exponenciais e logarítmicas',
'Conhecimentos algébricos - equações e inequações',
'Fenômenos Elétricos e Magnéticos - Relações entre grandezas elétricas: tensão, corrente, potência e energia'



----

In [167]:
linha=8
print(df_kc.iloc[linha].Aula)
print(df_kc.iloc[linha].Similarity)
df_kc.iloc[linha].KC

AULA_4__Perspectiva 
[0.775567673431277, 0.7568965773941023, 0.748937629741578, 0.7479342397190791, 0.7458591817794937, 0.7436190767811397, 0.7401975956190253, 0.7400197569917336, 0.7397565288493588, 0.7387076834507409]


[' Conhecimentos algébricos/geométricos  -   paralelismo e perpendicularidade, sistemas de equações.    ',
 ' Conhecimentos geométricos  -  características das figuras geométricas planas e  espaciais',
 ' Conhecimentos geométricos  -  simetrias de figuras planas ou espaciais',
 ' Conhecimentos geométricos  -  posições de retas',
 ' Conhecimentos geométricos  -  relações métricas nos  triângulos',
 ' Conhecimentos algébricos/geométricos  -  pla no cartesiano',
 ' Conhecimentos geométricos  -  trigonometria do ângulo agudo.  ',
 ' Conhecimentos geométricos  -  circunferências',
 ' Conhecimentos algébricos -   relações no ciclo trigonométrico e funções trigonométricas. ',
 ' Conhecimentos algébricos/geométricos  -  circunferências']

'Conhecimentos geométricos - características das figuras geométricas planas e espaciais' OK
'Conhecimentos geométricos - grandezas, unidades de medida e escalas' 
'Conhecimentos geométricos - comprimentos, áreas e volumes' 
'Conhecimentos geométricos - ângulos', 
'Conhecimentos geométricos - posições de retas', OK
'Conhecimentos algébricos/geométricos - plano cartesiano', OK
'Conhecimentos algébricos/geométricos - retas',
'Conhecimentos algébricos/geométricos - circunferências', OK
'Conhecimentos algébricos/geométricos - paralelismo e perpendicularidade, sistemas de equações. OK

In [183]:
kc_list = lista_kcs.KC.unique()
completion = llm.invoke(f"""
    System prompt: Act as an engineering professor which is responsible to combine his lectures to knowledge components.
    This task is important to register what knowledges each lecture is related and what is the pedagogical goal of each one.
    Context: The lecture you must analyze is between triple asterisks: ***{aula_texto_exemplo}***
    Context: The list of knowledge components goes betweem triple backticks ```{kc_list}```
    -------------------------------
    Output: Generate a list of knowledge components most related to the lecture provided
    Important: If the LLM can't find any related knowledge component the completion must be an empty list  
           """)

completion.content

"```['Conhecimentos numéricos - operações em conjuntos numéricos (naturais, inteiros, racionais e reais), desigualdades, divisibilidade, fatoração, razões e proporções, porcentagem e juros, relações de dependência entre grandezas, sequências e progressões, princípios de contagem.',\n 'Conhecimentos algébricos - gráficos e funções',\n 'Conhecimentos algébricos - funções algébricas do 1.º e do 2.º graus, polinomiais, racionais, exponenciais e logarítmicas',\n 'Conhecimentos algébricos - equações e inequações',\n 'Fenômenos Elétricos e Magnéticos - Relações entre grandezas elétricas: tensão, corrente, potência e energia']\n```"

In [168]:
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint="https://openai-dados-lab-poc.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_deployment="gpt-4o",
    openai_api_version="2024-02-01"
)

C:\Users\renan.vital\AppData\Local\Temp\ipykernel_11816\3739503124.py:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
